# Airbnb
El proyecto Inside Airbnb es un esfuerzo para obtener datos sobre propiedades disponibles en Airbnb con el objetivo de entender mejor el impacto que pueden tener en la disponibilidad y precio de vivienda en una ciudad. Analizaré los listados de viviendas disponibles en Ciudad de México para entender mejor el fenómeno y dar algunas recomendaciones.
## David Alejandro Guzmán Sánchez

Nota: importante antes de correr cualquier linea, correr primero las instalaciones plotly==4.9 y kaleido.

In [ ]:
# Instalación de librerias

!pip install -U plotly==4.9
!pip install -U kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.9 MB 6.7 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=3d9bbd93b4f27e5cf0f6b78b510069d7140fb5f56c5b80950c8e5c194cd7bb5b
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79.9 MB 105 kB/s 


# Descarga de datos

1. Usando Python descarga los datos de los archivos listings.csv para Ciudad de México
de todos los meses disponibles (no descarges el archivo listings.csv.gz tiene mucha
más info de la que necesitas)

In [ ]:
import pandas as pd
import requests

In [ ]:
#lista = ["22 March, 2021", "23 February, 2021", "29 January, 2021", "23 December, 2020", "27 November, 2020", "26 October, 2020", "20 June, 2020", "24 May, 2020"]

fechas = ["2021-03-22", "2021-02-23", "2021-01-29", "2020-12-23", "2020-11-27", "2020-10-26", "2020-06-20", "2020-05-24"]
df_final = pd.DataFrame() # df donde se guarda toda la información de todos los meses

for fecha in fechas:
    url = "http://data.insideairbnb.com/mexico/df/mexico-city/"+fecha+"/visualisations/listings.csv"
    response = requests.get(url)
    if response.status_code == 200: # el archivo está disponible
        df = pd.read_csv(url) # cargamos la información
        df["date"] = fecha # agregamos una columna con la fecha
        df_final = pd.concat([df_final, df], axis=0) # concatemos información

df_final = df_final.reset_index(drop=True) # reiniciamos índice

Dado que las información tiene datos repetidos en todos lo meses (pues suelen ser los mismos anuncios), para el análisis consideramos solo los datos del último mes (22 de marzo de 2021)

In [ ]:
fecha = "2021-03-22"
url = "http://data.insideairbnb.com/mexico/df/mexico-city/"+fecha+"/visualisations/listings.csv"
response = requests.get(url)
df = pd.read_csv(url)
df["date"] = fecha

# Análisis

Una de las preocupaciones principales de Airbnb es que se convierta en un mecanismo que desplace a residentes de la zona pues suele ser más rentable para un propietario poner sus propiedades en Airbnb que rentarlo a largo plazo agente local. En este sentido, muchos hosts actúan más como un negocio hotelero que como un individuo que ocasionalmente pone en Airbnb su casa cuando sale de viaje.

2. Supongamos que trabajas en el GDF y te piden hacer una propuesta para entender
mejor el fenómeno de Airbnb en CDMX con miras a determinar si se necesita establecer
mejores controles. Haz una presentación de no más de 5 láminas donde dados tu
análisis ayudes a entender mejor el fenómeno en CDMX y determines si es necesario
poner limitaciones en CDMX y en caso de sí, cuál sería tu propuesta? Algunos aspectos
a considerar (no te estoy pidiendo que consideres todo esto en tu presentación,
sólo son ideas para orientar tu análisis, te dejo a ti lo que quieras comunicar)



> a. Python hace gráficas más bonitas que Excel



> b. Se puede dar el caso que haya listings que sigan de alta pero que estén
“inactivos”

> c. Probablemente el impacto no sea igual por Alcaldía

> d. En el último año, dada la pandemia, Airbnb probablemente cambió mucho su
uso y composición

> e. Existen muchos tipos de hosts, por ejemplo, “hoteleros” (muchas propiedades,
disponibles todo el año) y “ocasionales” (una propiedad o cuarto, disponible
pocos días al año).

> f. El número de listados de Airbnb relativo al total disponible en CDMX o relativo a
otras ciudades podría indicarnos que tan urgente es tomar acciones (tampoco
queremos sobreregular)

## Porcentaje de tipos de habitación

Calculamos el porcentaje que ocupan los diferentes tipos de habitación

In [ ]:
import plotly.express as px
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/MyDrive/Colab Notebooks/Examen_Final/"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Tipo de habitación

df_room_type = pd.DataFrame(df["room_type"].value_counts()) # clasificamos valores de room_type y contamos cuantos hay de cada uno
df_room_type["proportion"] = df_room_type/sum(df_room_type.room_type) # sacamos proporción

# Gráfica
fig = px.pie(df_room_type, names=df_room_type.index, values=df_room_type.room_type, title="Tipo de habitación", hole=0.3)
fig.write_image(path+"tipos_habitación.png")
fig.show()
df_room_type


,room_type,proportion
Entire home/apt,10575,0.528195
Private room,8825,0.440787
Shared room,378,0.018880
Hotel room,243,0.012137


## Actividad registrada de marzo de 2020

Cálculamos indicadores que nos dan idea sobre la actividad de los anunciantes tales como:
1. noches estimadas = noches minímas * número de reseñas
2. Evaluaciones por mes = media de las evaluaciones por mes
3. Evaluaciones = suma del total de las evaluaciones
4. Precio = precio medio
5. Tasa de ocupación = noches estimadas / disponibilidad media
6. Beneficios = media de noches mínimas * número de evaluaciones * precio / 12

In [ ]:
# Actividad 

noches_estimadas = (df["minimum_nights"]*df["number_of_reviews"]).mean()
evaluaciones_mes = df["reviews_per_month"].mean()
evaluaciones = df["number_of_reviews"].sum()
precio = df["price"].mean()
tasa_ocupacion = noches_estimadas/df["availability_365"].mean() 
beneficios = (df["minimum_nights"]*df["number_of_reviews"]*df["price"]).mean()/12 

print(noches_estimadas)
print(evaluaciones_mes)
print(evaluaciones)
print(precio)
print(tasa_ocupacion)
print(beneficios)


62.276359822186706
1.2173939145858788
476732
1147.452175216023
0.2746001143474312
5806.617880392255


## Anuncios por cada anfitrión

El pilar del análisis se basa en detectar las alcaldias que ponen mas anuncios, pues aumenta la posibilidad de que las rentas tengan fin hotelero

In [ ]:
# Deleagaciones que tienen más propiedades en renta 
 
top_alcaldias = df.groupby("neighbourhood").size().sort_values(ascending=False) # contamos los anuncios por alcaldia y ordenamos

# Gráfica
fig = px.bar(top_alcaldias, x=top_alcaldias.index, y=top_alcaldias.values, title="Anuncios por alcaldia")
fig.update_traces(marker_color='rgb(400,70,30)', marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.6,)
fig.update_yaxes(title_text="<b>Cantidad de anuncios</b>") # Titulo en eje-y
fig.update_xaxes(title_text="<b>Alcaldia</b>") # Titulo en eje-x
fig.write_image(path+"Anuncios por alcadia.png")

fig.show()


Para detectar a los posibles hoteleros, podemos ver cuantos anuncios hace cada usuario mediante su host_id y detectar a los que mas anuncios tienen, luego tomamos un top

In [ ]:
# Anuncios por cada anfitrión

# Obtenemos los que tienen más anuncios fijandonos en el host_id
df_anuncios = df["host_id"].value_counts().sort_values(ascending=False)
# Tomamos el top-10
top_id = list(df_anuncios[0:10].index)
# Metemos el top-10 en un dataframe
top_host = df[df.host_id.isin(top_id)][["host_id", "host_name"]].drop_duplicates().set_index("host_id").reindex(top_id)
top_host["anuncios"] = df_anuncios[:10].values

# Gráfica
fig = px.bar(top, x=top_host.host_name, y=top_host.anuncios, title="Posibles aprovechados")
fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.6) # Decorado
fig.update_yaxes(title_text="<b>Cantidad de anuncios</b>") # Titulo en eje-y
fig.update_xaxes(title_text="<b>Usuario</b>") # Titulo en eje-x
fig.show()

top_host

,host_name,anuncios
host_id,,
22758914,Raquel,121
91265490,HOMi,109
236301634,Rosalia Dawn,79
368595849,Claudia,50
2281290,Suites,50
4296667,Andy,48
207240505,Luis,47
276377520,María Del Carmen,46
261857354,Mr. W,43


En el análisis podemos estudiar cuantos usuarios tienen multiples anuncios y cuantos tienen un solo anuncio. Lo anterior nos da una pauta del verdadeo uso que se le podría estar dando a Airbnb

In [ ]:

# Contamos los usuarios que tienen multiples anuncios
anuncios_multiples = (df_anuncios > 1).sum()
# Contamos los usuarios que tienen anuncios simples (sólo uno)
anuncios_simples = len(df_anuncios) - anuncios_multiples

# Hacemos datafrane para guardar lo calculado
df_anun_total = pd.DataFrame([anuncios_simples, anuncios_multiples], index=["anuncios simples", "anuncios multiples"], columns=["proportion"])

# Gráfica
fig = px.pie(df_anun_total, names=df_anun_total.index, values=df_anun_total.proportion, title="Usuarios con anuncios simples/multiples", hole=0.3)
fig.write_image(path+"anuncios_mul_sim.png")
fig.show()

df_anun_total

,proportion
anuncios simples,8432
anuncios multiples,3031


Note que la mayoria tiene anuncios simples, y el 26.4% pueden ser posibles aprovechados

In [ ]:
# Cantidad de anuncios publicados en los diferentes periodos

anuncios_por_fecha = df_final.groupby("date").size() # cantidad de anuncios por fecha

anuncios_por_fecha = pd.DataFrame(anuncios_por_fecha, columns=["Anuncios"]) # establecemos nombre de columna
anuncios_por_fecha

,Anuncios
date,
2020-06-20,21824
2020-10-26,19180
2020-11-27,19492
2020-12-23,19852
2021-01-29,19955
2021-02-23,20065
2021-03-22,20021


## Disponibilidad

Podriamos pensar que solo las propiedades tipo Entire home/apt son las que podrían desplazar gente local de la CDMX, sin embargo puede ser el caso en el que una persona ponga en renta una casa completa por habitaciones, lo que se le conoce como rentar con "roomies", este hecho lo ocuparemos más adeltante en nuestros análisis.

Definimos alta disponibilidad si la propiedad se renta más dos meses año, esto es, 60 días y entonces habría sospecha de que la propiedad se renta con fines de hotelería

In [ ]:
# Disponibilidad

# Alta disponibilidad (> 60)
alta_disp = len(df[df.availability_365>60])/len(df) # filtramos cuantos anuncios tienen alta disponibilidad
# Baja disponibilidad
baja_disp = 1 - alta_disp

# Hacemos dataframe con los datos calculados para gráficar
df_disp_total = pd.DataFrame([baja_disp, alta_disp], index=["baja disponibilidad", "alta_disponibilidad"], columns=["proportion"])

# Gráfica
fig = px.pie(df_disp_total, names=df_disp_total.index, values=df_disp_total.proportion, title="Disponibilidad", hole=0.3)
fig.write_image(path+"disponibilidad.png")
fig.show()

df_disp_total

,proportion
baja disponibilidad,0.160132
alta_disponibilidad,0.839868


Disponibilidad por casa/depto entero o por habitación privada

In [ ]:
# Disponibilidad agrupando por alcaldia y tipo de habitación
disponibilidad_room_type = df[df.neighbourhood.isin(top_anun_alcaldia)].groupby(["neighbourhood", "room_type"])["availability_365"].mean().reset_index()

# Disponibilidad por propiedad entera
dis_entire_room = disponibilidad_room_type[disponibilidad_room_type.room_type == "Entire home/apt"].sort_values("availability_365", ascending=False)
# Disponibilidad por habitación privada
dis_private_room = disponibilidad_room_type[disponibilidad_room_type.room_type == "Private room"].sort_values("availability_365", ascending=False)

# Gráficas
fig = px.pie(dis_entire_room, names= "neighbourhood", values= "availability_365", title="Disponibilidad Casa/Depto entero")
fig.show()
fig = px.pie(dis_private_room, names="neighbourhood", values="availability_365", title="Disponibilidad habitación privada")
fig.show()

In [ ]:
# Alta disponibilidad casa o depto entero
df_entire = df[df.room_type == "Entire home/apt"]
alta_discomp = df_entire[df_entire.availability_365>60]
disp_comp = len(alta_discomp)/len(df_entire)
disp_comp

# Note que la disponibilidad por casa o depto es similar a cuando se hace en el agregado de los 
# diferentes tipos de habitación

0.8465248226950355

Gráfica de número de días disponibles para rentar su propiedad al año, que ponen los usuarios de sus propiedades

In [ ]:
# Número de días disponibles al año

num_anun_disp = df.groupby("availability_365").size() # agrupamos por disponibilidad y contamos anuncios

# Gráfica
fig = px.bar(num_anun_disp, x=num_anun_disp.index, y=num_anun_disp.values, title="Días disponibles en los anuncios")
fig.update_xaxes(title_text="<b>número de días disponibles al año<b>") # Titulo en eje-x
fig.update_yaxes(title_text="<b>anuncios</b>") # Titulo en eje-y

# guardamos gráficas y mostramos
fig.write_image(path+"Días disponibles en los anuncios.png")
fig.show()

# Se puede ver que la mayoria pone sus casas en renta la mayor parte del año

Se ofrece una Gráfica de caja sobre la disponibilidad, para entender como se distribuye la disponibilidad de las propiedas en renta (incluyendo todos los tipos) en la CDMX através de los meses de pandemia.

Note que solo en el mes de noviembre hay un sesgo a la derecha.

In [ ]:
fig = px.box(df_final, x="date", y="availability_365", title="Disponibilidad por mes")
fig.write_image(path+"caja.png")
fig.show()

## Precio promedio de renta por alcaldia con mayores anuncios

In [ ]:
# top de las rentas mas caras por alcaldia

# Top-5 de las alcaldias con mayor número de anuncios
top_anun_alcaldia = top_alcaldias[:5].index.tolist()

# Precio promedio de renta por alcaldia con mayores anuncios
precio_rentas = df[df.neighbourhood.isin(top_anun_alcaldia)].groupby("neighbourhood")["price"].mean().sort_values(ascending=False)

# Gráfica
fig = px.bar(precio_rentas, x=precio_rentas.index, y=precio_rentas.values, title="Precio promedio de renta")
fig.update_traces(marker_color='rgb(120,252,25)', marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.6,)
fig.update_yaxes(title_text="<b>Precio $</b>") # Titulo en eje-y
fig.update_xaxes(title_text="<b>Alcaldia</b>") # Titulo en eje-x
fig.show()


## Mapa 

Como extra, ofrecemos un mapa de dispersión del tipo de vivienda que se renta en la CDMX

In [ ]:
# Gráfica de mapa
fig = px.scatter_mapbox(df[df.room_type.isin(["Entire home/apt", "Private room"])], lat="latitude", lon="longitude", color="room_type", hover_name="neighbourhood", zoom=9.3, opacity=.9)
fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}) # margenes
fig.write_image(path+"mapa.png")
fig.show()

# Conclusión

* El pilar del análisis se basa en detectar las alcaldías que ponen más anuncios, pues esto aumenta la posibilidad de que las rentas tengan fin hotelero​.
* Las alcaldías en donde más se publican anuncios es en: Cuauhtémoc, Miguel Hidalgo, Benito Juárez, Coyoacán​.
* Con los datos podemos observar que la mayoría de usuarios solo tiene un anuncio, mientras que la cuarta parte podría estar dando un uso hotelero​.
* La mayoría de viviendas que se rentan son casas enteras y habitaciones privadas.
* El que una propiedad tenga una alta disponibilidad para los turistas puede implicar que el propietario no esté presente, que sea ilegal y lo más importante, que suponga la expulsión de los residentes.​ Del análisis se obtiene que una gran parte de los anuncios indican que las propiedades están disponibles todo el año, lo cual es indicador de que se están rentando con propósito diferente al establecido.​
* Podríamos pensar que solo casas completas son las que podrían desplazar gente local de la CDMX, sin embargo, puede ser el caso en el que una persona ponga en renta una casa completa por habitaciones. Definimos alta disponibilidad si la propiedad se renta más dos meses año, esto es, 60 días y entonces habría sospecha de que la propiedad se renta con fines de hotelería.​
* Durante la pandemia, la distribución de la disponibilidad de las propiedad en renta en la CDMX a través de los meses de pandemia se mantuvo relativamente igual.​ Sólo en el mes de noviembre hay un sesgo a la derecha.​
* En efecto, se concluye que se está cometiendo desplazamiento de los ciudadanos en las alcaldías Miguel Hidalgo, Cuauhtémoc y Benito Juárez, ya que se están cobrando precios altos y las casas enteras se están rentando por habitación.